In [351]:
# Instalamos torchtext que nos facilitará la vida en el pre-procesamiento del formato ConLL.
# !pip install -U torchtext==0.10.0

# Librerias

In [352]:
import random
import os, shutil
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import gzip
import os
import shutil
import requests

from operator import attrgetter
from torchtext import vocab, datasets ,data
#from torchtext.legacy import data #, datasets
from seqeval.metrics import f1_score, precision_score, recall_score

In [353]:
# Garantizar reproducibilidad de los experimentos
SEED = 1234
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

#### **Obtener datos**

Descargamos los datos de entrenamiento, validación y prueba en nuestro directorio de trabajo

In [354]:
#%%capture

# !wget https://github.com/dccuchile/CC6205/releases/download/v1.0/train.txt -nc # Dataset de Entrenamiento
# !wget https://github.com/dccuchile/CC6205/releases/download/v1.0/dev.txt -nc    # Dataset de Validación (Para probar y ajustar el modelo)
# !wget https://github.com/dccuchile/CC6205/releases/download/v1.0/test.txt -nc  # Dataset de la Competencia. Estos datos solo contienen los tokens. ¡¡SON LOS QUE DEBEN SER PREDICHOS!!

In [355]:
# Primer Field: TEXT. Representan los tokens de la secuencia
TEXT = data.Field(lower=False) 

# Segundo Field: NER_TAGS. Representan los Tags asociados a cada palabra.
NER_TAGS = data.Field(unk_token=None)
fields = (("text", TEXT), ("nertags", NER_TAGS))

print(fields)

(('text', <torchtext.data.field.Field object at 0x7f8018eef0d0>), ('nertags', <torchtext.data.field.Field object at 0x7f8018eee1a0>))


####  **SequenceTaggingDataset**

`SequenceTaggingDataset` es una clase de torchtext diseñada para contener datasets de sequence labeling. Los ejemplos que se guarden en una instancia de estos serán arreglos de palabras asociados con sus respectivos tags.

Por ejemplo, para Part-of-speech tagging:

[I, love, PyTorch, .] estará asociado con [PRON, VERB, PROPN, PUNCT]


La idea es que usando los fields que definimos antes, le indiquemos a la clase cómo cargar los datasets de prueba, validación y test.

In [356]:
# train_data_ft, valid_data_ft, test_data_ft = datasets.SequenceTaggingDataset.splits(
#     path="./",
#     train="corpus_recetas_train.txt",
#     validation="corpus_recetas_val.txt",
#     test="corpus_recetas_test.txt",
#     fields=fields,
#     encoding="utf-8",
#     separator=" "
# )
# train_data_ft, valid_data_ft, test_data_ft = datasets.SequenceTaggingDataset.splits(
#     path="./",
#     train="corpus_train.txt",
#     validation="corpus_val.txt",
#     #test="corpus_test.txt",
#     test="corpus_test.txt",
#     fields=fields,
#     encoding="utf-8",
#     separator=" " 
# )
train_data_ft, valid_data_ft, test_data_ft = datasets.SequenceTaggingDataset.splits(
    path="./",
    train="corpus_train.txt",
    validation="corpus_val.txt",
    test="corpus_test_pred.txt",
    fields=fields,
    encoding="utf-8",
    separator="-X- _ " 
    # separator="-X-"
)

train_data, valid_data, test_data = datasets.SequenceTaggingDataset.splits(
    path="./",
    train="corpus_ER_train.txt",
    validation="corpus_ER_test.txt", # val y test son iguales pero no importa porque solo se usa val para entrenar, y luego se hace fine tuning con las variables ft
    test="corpus_ER_test.txt",
    fields=fields,
    encoding="utf-8",
    separator=" "
)

In [357]:
# valid_data = train_data[int(len(train_data)*0.7):int(len(train_data)*0.8)]
# test_data = train_data[int(len(train_data)*0.8):len(train_data)]
# train_data = train_data[0:int(len(train_data)*0.7)]

In [358]:
print(f"Numero de ejemplos de entrenamiento: {len(train_data)}")
print(f"Número de ejemplos de validación: {len(valid_data)}")
print(f"Número de ejemplos de test: {len(test_data)}")

Numero de ejemplos de entrenamiento: 85245
Número de ejemplos de validación: 51182
Número de ejemplos de test: 51182


Visualizemos un ejemplo

In [359]:
random_item_idx = random.randint(0, len(train_data))
random_example = train_data.examples[random_item_idx]
list(zip(random_example.text, random_example.nertags))

[('CLOBETASOL', 'B-ACTIVE_PRINCIPLE'),
 ('0,05', 'O'),
 ('%', 'O'),
 ('UNGÜENTO', 'B-FORMA_FARMA'),
 ('DÉRMICO', 'O'),
 ('TUBO', 'O'),
 ('30', 'O'),
 ('G', 'O'),
 ('1', 'O'),
 ('APLICACIÓN', 'O'),
 ('USO', 'O'),
 ('CUTÁNEO', 'B-ADMIN'),
 ('dos', 'B-PERIODICITY'),
 ('veces', 'I-PERIODICITY'),
 ('al', 'I-PERIODICITY'),
 ('día', 'I-PERIODICITY')]

#### **Construir los vocabularios para el texto y las etiquetas**

Los vocabularios son los objetos que contienen todos los tokens (de entrenamiento) posibles para ambos fields. El siguiente paso consiste en construirlos. Para esto, hacemos uso del método `Field.build_vocab` sobre cada uno de nuestros `fields`. 

In [360]:
TEXT.build_vocab(train_data)
NER_TAGS.build_vocab(train_data,train_data_ft)
# NER_TAGS.build_vocab(train_data_ft)

In [361]:
print(f"Tokens únicos en TEXT: {len(TEXT.vocab)}")
print(f"Tokens únicos en NER_TAGS: {len(NER_TAGS.vocab)}")

Tokens únicos en TEXT: 5678
Tokens únicos en NER_TAGS: 12


In [362]:
#Veamos las posibles etiquetas que hemos cargado:
NER_TAGS.vocab.itos

['<pad>',
 'O',
 'I-PERIODICITY',
 'B-PERIODICITY',
 'B-ACTIVE_PRINCIPLE',
 'B-FORMA_FARMA',
 'B-ADMIN',
 'I-FORMA_FARMA',
 'I-DURATION',
 'B-DURATION',
 'I-ACTIVE_PRINCIPLE',
 'I-ADMIN']

Observen que ademas de los tags NER, tenemos \<pad\>, el cual es generado por el dataloader para cumplir con el padding de cada oración.

Veamos ahora los tokens mas frecuentes y especiales:

In [363]:
# Tokens mas frecuentes (Será necesario usar stopwords, eliminar símbolos o nos entregan información (?) )
TEXT.vocab.freqs.most_common(10)

[('MG', 61740),
 ('ML', 55473),
 ('cada', 50536),
 ('horas', 47976),
 ('1', 44157),
 ('ORAL', 44050),
 ('AMPOLLA', 32629),
 ('solución', 30173),
 ('comprimido', 28550),
 ('FRASCO', 28149)]

In [364]:
# Seteamos algunas variables que nos serán de utilidad mas adelante...
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

PAD_TAG_IDX = NER_TAGS.vocab.stoi[NER_TAGS.pad_token]
O_TAG_IDX = NER_TAGS.vocab.stoi['O']
# O_TAG_IDX2 = NER_TAGS.vocab.stoi[' O']
#O_TAG_IDX = NER_TAGS.vocab.stoi['O']

#### **Frecuencia de los Tags**

Visualizemos rápidamente las cantidades y frecuencias de cada tag:

In [365]:
def tag_percentage(tag_counts):
    
    total_count = sum([count for tag, count in tag_counts])
    tag_counts_percentages = [(tag, count, count/total_count) for tag, count in tag_counts]
  
    return tag_counts_percentages

print("Tag Ocurrencia Porcentaje\n")

for tag, count, percent in tag_percentage(NER_TAGS.vocab.freqs.most_common()):
    print(f"{tag}\t{count}\t{percent*100:4.1f}%")

Tag Ocurrencia Porcentaje

O	703243	54.1%
I-PERIODICITY	130090	10.0%
B-PERIODICITY	80764	 6.2%
B-ACTIVE_PRINCIPLE	75608	 5.8%
B-FORMA_FARMA	74462	 5.7%
B-ADMIN	74023	 5.7%
I-FORMA_FARMA	73811	 5.7%
I-DURATION	45030	 3.5%
B-DURATION	22278	 1.7%
I-ACTIVE_PRINCIPLE	19889	 1.5%
I-ADMIN	1380	 0.1%


#### **Configuramos pytorch y dividimos los datos.**

Importante: si tienes problemas con la ram de la gpu, disminuye el tamaño de los batches

In [366]:
BATCH_SIZE = 32 #16  # disminuir si hay problemas de ram.

# Usar cuda si es que está disponible.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using', device)

# Dividir datos entre entrenamiento y test. Si van a hacer algún sort no puede ser sobre
# el conjunto de testing ya que al hacer sus predicciones sobre el conjunto de test sin etiquetas
# debe conservar el orden original para ser comparado con los golden_labels. 

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size=BATCH_SIZE,
    device=device,
    sort=False,
)

train_iterator_ft, valid_iterator_ft, test_iterator_ft = data.BucketIterator.splits(
    (train_data_ft, valid_data_ft, test_data_ft),
    batch_size=BATCH_SIZE,
    device=device,
    sort=False,
)

# test_loader_df =  pd.read_csv("corpus_test.txt", sep="\n", header=None)
# test_loader = torch.utils.data.DataLoader(test_loader_df, batch_size=BATCH_SIZE)

Using cuda


#### **Métricas de evaluación**

Además, definiremos las métricas que serán usadas tanto para la competencia como para evaluar el modelo: `precision`, `recall` y `micro f1-score`.
**Importante**: Noten que la evaluación solo se hace para las Named Entities (sin contar 'O'), toda esta funcionalidad nos la entrega la librería seqeval, pueden revisar más documentación aquí: https://github.com/chakki-works/seqeval. No utilicen el código entregado por sklearn para calcular las métricas ya que esta lo hace a nivel de token y no a nivel de entidad.

In [367]:
# Definimos las métricas
def calculate_metrics(preds, y_true, pad_idx=PAD_TAG_IDX, o_idx=O_TAG_IDX):
    """
    Calcula precision, recall y f1 de cada batch.
    """

    # Obtener el indice de la clase con probabilidad mayor. (clases)
    y_pred = preds.argmax(dim=1, keepdim=True)

    # filtramos <pad> para calcular los scores.
    mask = [(y_true != pad_idx)]
    y_pred = y_pred[mask]
    y_true = y_true[mask]

    # traemos a la cpu
    y_pred = y_pred.view(-1).to('cpu').numpy()
    y_true = y_true.to('cpu').numpy()
    y_pred = [[NER_TAGS.vocab.itos[v] for v in y_pred]]
    y_true = [[NER_TAGS.vocab.itos[v] for v in y_true]]
    
    # calcular scores
    f1 = f1_score(y_true, y_pred, mode='strict')
    precision = precision_score(y_true, y_pred, mode='strict')
    recall = recall_score(y_true, y_pred, mode='strict')

    return precision, recall, f1

-------------------

### **Modelo Baseline**

Teniendo ya cargado los datos, toca definir nuestro modelo. Este baseline tendrá una capa de embedding, unas cuantas LSTM y una capa de salida y usará dropout en el entrenamiento.

Este constará de los siguientes pasos: 

1. Definir la clase que contendrá la red.
2. Definir los hiperparámetros e inicializar la red. 
3. Definir el número de épocas de entrenamiento
4. Definir la función de loss.



Recomendamos que para experimentar, encapsules los modelos en una sola variable y luego la fijes en model para entrenarla

In [368]:
# Definir la red
class NER_RNN(nn.Module):
    def __init__(self, 
                 input_dim, 
                 embedding_dim, 
                 hidden_dim, 
                 output_dim,
                 n_layers, 
                 bidirectional, 
                 dropout, 
                 pad_idx):

        super().__init__()

        # Capa de embedding
        self.embedding = nn.Embedding(input_dim,
                                      embedding_dim,
                                      padding_idx=pad_idx,
                                      )

        # Capa LSTM
        self.lstm = nn.LSTM(embedding_dim,
                           hidden_dim,
                           num_layers=n_layers,
                           bidirectional=bidirectional, 
                           dropout = dropout if n_layers > 1 else 0)

        # Capa de salida
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim,
                            output_dim)

        # Dropout
        self.dropout = nn.Dropout(dropout)

    def forward(self, text):

        #text = [sent len, batch size]

        # Convertir lo enviado a embedding
        embedded = self.dropout(self.embedding(text))
        
        outputs, (hidden, cell) = self.lstm(embedded)
        #embedded = [sent len, batch size, emb dim]

        # Pasar los embeddings por la rnn (LSTM)

        #output = [sent len, batch size, hid dim * n directions]
        #hidden/cell = [n layers * n directions, batch size, hid dim]

        # Predecir usando la capa de salida.
        predictions = self.fc(self.dropout(outputs))
        #predictions = [sent len, batch size, output dim]

        return predictions

#### **Hiperparámetros de la red**



In [369]:
# tamaño del vocabulario. recuerden que la entrada son vectores bag of word(one-hot).
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300  # dimensión de los embeddings.
HIDDEN_DIM = 256  # dimensión de la capas LSTM
OUTPUT_DIM = len(NER_TAGS.vocab)  # número de clases

N_LAYERS = 3  # número de capas.
DROPOUT = 0.5
BIDIRECTIONAL = False

# Creamos nuestro modelo.
baseline_model = NER_RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM,
                         N_LAYERS, BIDIRECTIONAL, DROPOUT, PAD_IDX)

baseline_model_name = 'baseline'  # nombre que tendrá el modelo guardado...

In [370]:
baseline_n_epochs = 10

#### Definimos la función de loss

In [371]:
# Loss: Cross Entropy
TAG_PAD_IDX = NER_TAGS.vocab.stoi[NER_TAGS.pad_token]
baseline_criterion = nn.CrossEntropyLoss(ignore_index = TAG_PAD_IDX)

--------------------
### Modelo 1

En estas secciones pueden implementar nuevas redes al modificar los hiperparámetros, la cantidad de épocas de entrenamiento, el tamaño de los batches, loss, optimizador, etc... como también definir nuevas arquitecturas de red (mediante la creación de clases nuevas)


Al final de estas, hay 4 variables, las cuales deben setear con los modelos, épocas de entrenamiento, loss y optimizador que deseen probar.


In [372]:
# Caergamos Glove o fast text
FASTTEXT_FILE = "glove300d.vec"
# Se descargan vectores glove o fasttext del github del dcc
# https://github.com/dccuchile/spanish-word-embeddings

if not os.path.exists(FASTTEXT_FILE):
    print(f"Descargando {FASTTEXT_FILE}")
    url = "http://dcc.uchile.cl/~jperez/word-embeddings/glove-sbwc.i25.vec.gz"
    #url = "https://s06.imfd.cl/04/fasttext-sbwc.vec.gz"
    response = requests.get(url, stream=True)
    try:
        with gzip.open(response.raw, "rb") as f_in:
            with open(FASTTEXT_FILE, "wb") as f_out:
                # Funcion para copiar de un file-like object a otro
                shutil.copyfileobj(f_in, f_out)
    except Exception as e:
        os.remove(FASTTEXT_FILE)
        raise e

 #dimensión es de 300 y tiene 855,380 vectores pre-entrenados. 

In [373]:
embeddings = vocab.Vectors(FASTTEXT_FILE)


In [374]:
TEXT.vocab.set_vectors(*attrgetter("stoi", "vectors", "dim")(embeddings))
#se asocian los tokens de text a los vectores del embedding


In [375]:
# BATCH_SIZE = 32

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# print('Using', device)

# train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
#     (train_data, valid_data, test_data),
#     batch_size=BATCH_SIZE,
#     device=device,
#     sort=False,
# )

In [376]:
class Modelo1_RNN(nn.Module):
    def __init__(self, 
                 input_dim, 
                 embedding_dim, 
                 hidden_dim, 
                 output_dim,
                 n_layers, 
                 bidirectional, 
                 dropout, 
                 pad_idx):

        super().__init__()

        # Capa de embedding
        self.embedding = nn.Embedding.from_pretrained(
            embedding_weights.clone(), freeze = False)
        # Capa LSTM
        self.lstm = nn.LSTM(embedding_dim,
                           hidden_dim,
                           num_layers=n_layers,
                           bidirectional=bidirectional, 
                           dropout = dropout if n_layers > 1 else 0)

        # Capa de salida
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim,
                            output_dim)

        # Dropout
        self.dropout = nn.Dropout(dropout)
        

    def forward(self, text):

        #text = [sent len, batch size]

        # Convertir lo enviado a embedding
        embedded = self.dropout(self.embedding(text))
        
        outputs, (hidden, cell) = self.lstm(embedded)
        #embedded = [sent len, batch size, emb dim]

        # Pasar los embeddings por la rnn (LSTM)

        #output = [sent len, batch size, hid dim * n directions]
        #hidden/cell = [n layers * n directions, batch size, hid dim]

        # Predecir usando la capa de salida.
        predictions = self.fc(self.dropout(outputs))
        #predictions = [sent len, batch size, output dim]

        return predictions

In [377]:
#Parametros para el modelo 1
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
HIDDEN_DIM = 196
OUTPUT_DIM = len(NER_TAGS.vocab)
embedding_weights = TEXT.vocab.vectors

N_LAYERS = 3
DROPOUT = 0.15
BIDIRECTIONAL = True

TAG_PAD_IDX = NER_TAGS.vocab.stoi[NER_TAGS.pad_token]


In [378]:
modelo_1 = Modelo1_RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, 
                         N_LAYERS, BIDIRECTIONAL, DROPOUT, PAD_IDX)

model_name_1 = 'Modelo_1'
n_epochs_1 = 100 #no importa porq hay early stop
criterion_1 = nn.CrossEntropyLoss(ignore_index = TAG_PAD_IDX)

---------------

### Modelo 2

In [379]:
# BATCH_SIZE = 62 # disminuir si hay problemas de ram.

# # Usar cuda si es que está disponible.
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# print('Using', device)

# # Dividir datos entre entrenamiento y test. Si van a hacer algún sort no puede ser sobre
# # el conjunto de testing ya que al hacer sus predicciones sobre el conjunto de test sin etiquetas
# # debe conservar el orden original para ser comparado con los golden_labels. 

# train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
#     (train_data, valid_data, test_data),
#     batch_size=BATCH_SIZE,
#     device=device,
#     sort=False,
# )

In [380]:
class Modelo2_RNN(nn.Module):
    def __init__(self, 
                 input_dim, 
                 embedding_dim, 
                 hidden_dim, 
                 output_dim,
                 n_layers, 
                 bidirectional, 
                 dropout, 
                 pad_idx):

        super().__init__()

        # Capa de embedding
        self.embedding = nn.Embedding.from_pretrained(embedding_weights.clone(), freeze = False)

        # Capa LSTM
        self.lstm = nn.LSTM(embedding_dim,
                           hidden_dim,
                           num_layers=n_layers,
                           bidirectional=bidirectional, 
                           dropout = dropout if n_layers > 1 else 0)

        # Capa de salida
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim,
                            output_dim)
        #self.relu = nn.ReLU()

        # Dropout
        self.dropout = nn.Dropout(dropout)

    def forward(self, text):

        #text = [sent len, batch size]

        # Convertir lo enviado a embedding
        embedded = self.dropout(self.embedding(text))
        
        outputs, (hidden, cell) = self.lstm(embedded)
        #embedded = [sent len, batch size, emb dim]

        # Pasar los embeddings por la rnn (LSTM)

        #output = [sent len, batch size, hid dim * n directions]
        #hidden/cell = [n layers * n directions, batch size, hid dim]

        # Predecir usando la capa de salida.
        predictions = self.fc(self.dropout(outputs))
        #predictions = [sent len, batch size, output dim]

        return predictions

In [381]:
# tamaño del vocabulario. recuerden que la entrada son vectores bag of word(one-hot).
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300  # dimensión de los embeddings.
HIDDEN_DIM = 196  # dimensión de la capas LSTM
OUTPUT_DIM = len(NER_TAGS.vocab)  # número de clases

N_LAYERS = 2  # número de capas.
DROPOUT = 0.4
BIDIRECTIONAL = True
TAG_PAD_IDX = NER_TAGS.vocab.stoi[NER_TAGS.pad_token]

In [382]:
modelo_2 = Modelo2_RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, 
                         N_LAYERS, BIDIRECTIONAL, DROPOUT, PAD_IDX)

model_name_2 = 'Modelo_2'
n_epochs_2 = 30 #no importa porq hay early stop
criterion_2 = nn.CrossEntropyLoss(ignore_index = TAG_PAD_IDX)

---------------


### Modelo 3

In [383]:
# Para crear la red debemos heredar desde nn.Module
class GruNet(nn.Module):
    def __init__(self, 
                 input_dim, 
                 embedding_dim, 
                 hidden_dim, 
                 output_dim,
                 n_layers, 
                 bidirectional, 
                 dropout, 
                 pad_idx):

      super().__init__()

      # Capa de embedding
      self.embedding = nn.Embedding(input_dim,
                                    embedding_dim,
                                    padding_idx=pad_idx,
                                    )
      
      
      # Capa GRU
      self.gru = nn.GRU(embedding_dim, hidden_dim, n_layers, batch_first=True, dropout = dropout if n_layers > 1 else 0, bidirectional=bidirectional)

      # Capa de salida
      self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)


      #self.relu = nn.ReLU()

      # Dropout
      self.dropout = nn.Dropout(dropout)




    # Definimos las operaciones de las capas sobre el input en el forward.
    def forward(self, text): 

      embedded = self.embedding(text)

      outputs, hidden = self.gru(embedded)

      predictions = self.fc(self.dropout(outputs))


      return predictions


In [384]:
# tamaño del vocabulario. recuerden que la entrada son vectores bag of word(one-hot).
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300  # dimensión de los embeddings.
HIDDEN_DIM = 256  # dimensión de la capas LSTM
OUTPUT_DIM = len(NER_TAGS.vocab)  # número de clases

N_LAYERS = 2  # número de capas.
DROPOUT = 0.3
BIDIRECTIONAL = False

# Creamos nuestro modelo.
modelo_3 = GruNet(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM,
                         N_LAYERS, BIDIRECTIONAL, DROPOUT, PAD_IDX)

model_name_3 = 'Gru_Model'  # nombre que tendrá el modelo guardado...

n_epochs_3 = 10


# Loss: Cross Entropy
TAG_PAD_IDX = NER_TAGS.vocab.stoi[NER_TAGS.pad_token]
criterion_3 = nn.CrossEntropyLoss(ignore_index = TAG_PAD_IDX)



In [385]:
a ='123'
a[:2]

'12'

In [386]:
def function(fecha1,fecha2):
    Año_1=int(str(fecha1)[:2])
    Mes_1=int(str(fecha1)[-2:])
    Año_2=int(str(fecha2)[:2])
    Mes_2=int(str(fecha2)[-2:])

    dif=12*(Año_2-Año_1)+(Mes_2-Mes_1)
    return dif 



#### **Inicializamos la red**

Iniciamos los pesos de la red de forma aleatoria (Usando una distribución normal).


In [387]:
model = baseline_model
model_name = baseline_model_name
criterion = baseline_criterion
n_epochs = baseline_n_epochs

# Optimizador
optimizer = optim.Adam(model.parameters())

# # Enviamos el modelo y la loss a cuda (en el caso en que esté disponible)
# model = model.to(device)
# criterion = criterion.to(device)

In [388]:
def init_weights(m):
    # Inicializamos los pesos como aleatorios
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean=0, std=0.1) 
        
    # Seteamos como 0 los embeddings de UNK y PAD.
    model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
    model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)
        
model.apply(init_weights)

NER_RNN(
  (embedding): Embedding(5678, 300, padding_idx=1)
  (lstm): LSTM(300, 256, num_layers=3, dropout=0.5)
  (fc): Linear(in_features=256, out_features=12, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [389]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'El modelo actual tiene {count_parameters(model):,} parámetros entrenables.')

El modelo actual tiene 3,330,548 parámetros entrenables.


Notar que definimos los embeddings que representan a \<unk\> y \<pad\>  como [0, 0, ..., 0]

#### **Definimos el entrenamiento de la red**


In [390]:
def train(model, iterator, optimizer, criterion, otag=O_TAG_IDX):
    epoch_loss = 0
    epoch_precision = 0
    epoch_recall = 0
    epoch_f1 = 0

    model.train()
    # Por cada batch del iterador de la época:
    for batch in iterator:

        # Extraemos el texto y los tags del batch que estamos procesado
        text = batch.text
        tags = batch.nertags

        # Reiniciamos los gradientes calculados en la iteración anterior
        optimizer.zero_grad()

        #text = [sent len, batch size]

        # Predecimos los tags del texto del batch.
        predictions = model(text)

        #predictions = [sent len, batch size, output dim]
        #tags = [sent len, batch size]

        # Reordenamos los datos para calcular la loss
        predictions = predictions.view(-1, predictions.shape[-1])
        tags = tags.view(-1)

        #predictions = [sent len * batch size, output dim]
        # Calculamos el Cross Entropy de las predicciones con respecto a las etiquetas reales
        loss = criterion(predictions, tags)
        
        # Calculamos el accuracy
        precision, recall, f1 = calculate_metrics(predictions, tags, o_idx=otag)

        # Calculamos los gradientes
        loss.backward()

        # Actualizamos los parámetros de la red
        optimizer.step()

        # Actualizamos el loss y las métricas
        epoch_loss += loss.item()
        epoch_precision += precision
        epoch_recall += recall
        epoch_f1 += f1

    return epoch_loss / len(iterator), epoch_precision / len(
        iterator), epoch_recall / len(iterator), epoch_f1 / len(iterator)

#### **Definimos la función de evaluación**

In [391]:
def evaluate(model, iterator, criterion, otag=O_TAG_IDX):

    epoch_loss = 0
    epoch_precision = 0
    epoch_recall = 0
    epoch_f1 = 0

    model.eval()

    # Indicamos que ahora no guardaremos los gradientes
    with torch.no_grad():
        # Por cada batch
        for batch in iterator:

            text = batch.text
            tags = batch.nertags

            # Predecimos
            predictions = model(text)

            predictions = predictions.view(-1, predictions.shape[-1])
            tags = tags.view(-1)

            # Calculamos el Cross Entropy de las predicciones con respecto a las etiquetas reales
            loss = criterion(predictions, tags)

            # Calculamos las métricas
            precision, recall, f1 = calculate_metrics(predictions, tags, o_idx=otag)

            # Actualizamos el loss y las métricas
            epoch_loss += loss.item()
            epoch_precision += precision
            epoch_recall += recall
            epoch_f1 += f1

    return epoch_loss / len(iterator), epoch_precision / len(
        iterator), epoch_recall / len(iterator), epoch_f1 / len(iterator)

In [392]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs


#### **Entrenamiento de la red**

En este cuadro de código ejecutaremos el entrenamiento de la red.
Para esto, primero definiremos el número de épocas y luego por cada época, ejecutaremos `train` y `evaluate`.

**Importante: Reiniciar los pesos del modelo**

Si ejecutas nuevamente esta celda, se seguira entrenando el mismo modelo una y otra vez. 
Para reiniciar el modelo se debe ejecutar nuevamente la celda que contiene la función `init_weights`



In [393]:
def train_model(model, model_name, train_iterator, valid_iterator, criterion, optimizer, n_epochs, otag=O_TAG_IDX):
  global device

  model = model.to(device)
  criterion = criterion.to(device)
  
  #Agregar early stop
  best_valid_loss = float('inf')
  best_train_loss = float('inf')

  arrayTrainLoss = []
  arrayValidLoss = []

  for epoch in range(n_epochs):

    start_time = time.time()

    # Recuerdo: train_iterator y valid_iterator contienen el dataset dividido en batches.

    # Entrenar
    train_loss, train_precision, train_recall, train_f1 = train(
        model, train_iterator, optimizer, criterion, otag)

    # Evaluar (valid = validación)
    valid_loss, valid_precision, valid_recall, valid_f1 = evaluate(
        model, valid_iterator, criterion, otag)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    #Nuevo **
    arrayTrainLoss.append(train_loss)
    arrayValidLoss.append(valid_loss)

  #Acá nos aseguramos de que entrene al menos 6 épocas antes de detener el entrenamiento.
    if epoch < 7:
      if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        #nuevo
        best_train_loss = train_loss
        torch.save(model.state_dict(), '{}.pt'.format(model_name))
        print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
        print(
            f'\tTrain Loss: {train_loss:.3f} | Train f1: {train_f1:.2f} | Train precision: {train_precision:.2f} | Train recall: {train_recall:.2f}'
        )
        print(
            f'\t Val. Loss: {valid_loss:.3f} |  Val. f1: {valid_f1:.2f} |  Val. precision: {valid_precision:.2f} | Val. recall: {valid_recall:.2f}'
        )
      else :
        print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
        print(
            f'\tTrain Loss: {train_loss:.3f} | Train f1: {train_f1:.2f} | Train precision: {train_precision:.2f} | Train recall: {train_recall:.2f}'
        )
        print(
            f'\t Val. Loss: {valid_loss:.3f} |  Val. f1: {valid_f1:.2f} |  Val. precision: {valid_precision:.2f} | Val. recall: {valid_recall:.2f}'
        )

  #Early stop, ve que no vaya en aumento, según el avg de las últimas 4 épocas
    else:
      if train_loss < np.mean(arrayTrainLoss[-4:]) and valid_loss > np.mean(arrayValidLoss[-4:]):
        print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
        print(
            f'\tTrain Loss: {train_loss:.3f} | Train f1: {train_f1:.2f} | Train precision: {train_precision:.2f} | Train recall: {train_recall:.2f}'
        )
        print(
            f'\t Val. Loss: {valid_loss:.3f} |  Val. f1: {valid_f1:.2f} |  Val. precision: {valid_precision:.2f} | Val. recall: {valid_recall:.2f}'
        )
        print('Early Stop')
        break

      else:
        best_valid_loss = valid_loss
        #nuevo
        best_train_loss = train_loss
        torch.save(model.state_dict(), '{}.pt'.format(model_name))
        print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
        print(
            f'\tTrain Loss: {train_loss:.3f} | Train f1: {train_f1:.2f} | Train precision: {train_precision:.2f} | Train recall: {train_recall:.2f}'
        )
        print(
            f'\t Val. Loss: {valid_loss:.3f} |  Val. f1: {valid_f1:.2f} |  Val. precision: {valid_precision:.2f} | Val. recall: {valid_recall:.2f}'
        )
  return model, arrayTrainLoss, arrayValidLoss

## Entrenamiento Baseline

In [394]:
model = baseline_model
model_name = baseline_model_name
model_name_ft = 'baseline_ft'
criterion = baseline_criterion
n_epochs = baseline_n_epochs

model, arrayTrainLoss, arrayValidLoss = train_model(model, model_name, train_iterator, valid_iterator, criterion, optimizer, n_epochs)
model_ft, arrayTrainLoss_ft, arrayValidLoss_ft = train_model(model, model_name_ft, train_iterator_ft, valid_iterator_ft, criterion, optimizer, n_epochs, otag=O_TAG_IDX)

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: <pad> seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


Epoch: 01 | Epoch Time: 0m 34s
	Train Loss: 0.073 | Train f1: 0.96 | Train precision: 0.96 | Train recall: 0.97
	 Val. Loss: 0.022 |  Val. f1: 0.99 |  Val. precision: 0.99 | Val. recall: 0.99
Epoch: 02 | Epoch Time: 0m 33s
	Train Loss: 0.022 | Train f1: 0.99 | Train precision: 0.98 | Train recall: 0.99
	 Val. Loss: 0.019 |  Val. f1: 0.99 |  Val. precision: 0.98 | Val. recall: 1.00
Epoch: 03 | Epoch Time: 0m 35s
	Train Loss: 0.020 | Train f1: 0.99 | Train precision: 0.98 | Train recall: 0.99
	 Val. Loss: 0.019 |  Val. f1: 0.99 |  Val. precision: 0.99 | Val. recall: 0.99
Epoch: 04 | Epoch Time: 0m 33s
	Train Loss: 0.019 | Train f1: 0.99 | Train precision: 0.98 | Train recall: 0.99
	 Val. Loss: 0.019 |  Val. f1: 0.99 |  Val. precision: 0.98 | Val. recall: 1.00
Epoch: 05 | Epoch Time: 0m 34s
	Train Loss: 0.019 | Train f1: 0.99 | Train precision: 0.99 | Train recall: 0.99
	 Val. Loss: 0.019 |  Val. f1: 0.99 |  Val. precision: 0.99 | Val. recall: 0.99
Epoch: 06 | Epoch Time: 0m 32s
	Train Lo

In [395]:
# cargar el mejor modelo entrenado.
model.load_state_dict(torch.load('{}.pt'.format(model_name_ft)))

# Limpiar ram de cuda
torch.cuda.empty_cache()

In [396]:
valid_loss, valid_precision, valid_recall, valid_f1 = evaluate(model_ft, valid_iterator_ft, criterion)
# test_loss, test_precision, test_recall, test_f1 = evaluate(model_ft, test_iterator, criterion)

print(f'Val. Loss: {valid_loss:.3f} |  Val. f1: {valid_f1:.2f} | Val. precision: {valid_precision:.2f} | Val. recall: {valid_recall:.2f}')
# print(f'test_loss: {test_loss:.3f} |  test_f1: {test_f1:.2f} | test_precision: {test_precision:.2f} | test_recall: {test_recall:.2f}')

Val. Loss: 1.318 |  Val. f1: 0.30 | Val. precision: 0.80 | Val. recall: 0.19


## Entrenamiento Modelo 1

In [ ]:
model1 = modelo_1
model_name1 = model_name_1
criterion1 = criterion_1
n_epochs1 = n_epochs_1

# Optimizador
optimizer1 = optim.Adam(model.parameters())
model1, arrayTrainLoss1, arrayValidLoss1 = train_model(model1, model_name1, criterion1, optimizer1, n_epochs1)

In [ ]:
# cargar el mejor modelo entrenado.
model1.load_state_dict(torch.load('{}.pt'.format(model_name1)))

# Limpiar ram de cuda
torch.cuda.empty_cache()

In [ ]:
valid_loss, valid_precision, valid_recall, valid_f1 = evaluate(model1, valid_iterator, criterion)
test_loss, test_precision, test_recall, test_f1 = evaluate(model1, test_iterator, criterion)

print(f'Val. Loss: {valid_loss:.3f} |  Val. f1: {valid_f1:.2f} | Val. precision: {valid_precision:.2f} | Val. recall: {valid_recall:.2f}')
print(f'test_loss: {test_loss:.3f} |  test_f1: {test_f1:.2f} | test_precision: {test_precision:.2f} | test_recall: {test_recall:.2f}')

## Entrenamiento Modelo 2

In [ ]:
model2 = modelo_2
model_name2 = model_name_2
criterion2 = criterion_2
n_epochs2 = n_epochs_2

# Optimizador
#optimizer = optim.Adam(model.parameters(),lr=0.3)
optimizer2 = optim.SGD(model.parameters(), lr=0.3, momentum=0.9)

model2, arrayTrainLoss2, arrayValidLoss2 = train_model(model2, model_name2, criterion2, optimizer2, n_epochs2)

In [ ]:
model2.load_state_dict(torch.load('{}.pt'.format(model_name2)))

# Limpiar ram de cuda
torch.cuda.empty_cache()

In [ ]:
valid_loss, valid_precision, valid_recall, valid_f1 = evaluate(model2, valid_iterator, criterion)
test_loss, test_precision, test_recall, test_f1 = evaluate(model2, test_iterator, criterion)

print(f'Val. Loss: {valid_loss:.3f} |  Val. f1: {valid_f1:.2f} | Val. precision: {valid_precision:.2f} | Val. recall: {valid_recall:.2f}')
print(f'test_loss: {test_loss:.3f} |  test_f1: {test_f1:.2f} | test_precision: {test_precision:.2f} | test_recall: {test_recall:.2f}')

## Entrenamiento Modelo 3

In [ ]:
model3 = modelo_3
model_name3 = model_name_3
criterion3 = criterion_3
n_epochs3 = n_epochs_3

# Optimizador
optimizer3 = optim.Adam(model3.parameters())

model3, arrayTrainLoss3, arrayValidLoss3 = train_model(model3, model_name3, criterion3, optimizer3, n_epochs3)

In [ ]:
model3.load_state_dict(torch.load('{}.pt'.format(model_name3)))

# Limpiar ram de cuda
torch.cuda.empty_cache()

In [ ]:
valid_loss, valid_precision, valid_recall, valid_f1 = evaluate(model3, valid_iterator, criterion)
test_loss, test_precision, test_recall, test_f1 = evaluate(model3, test_iterator, criterion)

print(f'Val. Loss: {valid_loss:.3f} |  Val. f1: {valid_f1:.2f} | Val. precision: {valid_precision:.2f} | Val. recall: {valid_recall:.2f}')
print(f'test_loss: {test_loss:.3f} |  test_f1: {test_f1:.2f} | test_precision: {test_precision:.2f} | test_recall: {test_recall:.2f}')

### **Predecir datos para la competencia**

Ahora, a partir de los datos de **test** y nuestro modelo entrenado, vamos a predecir las etiquetas que serán evaluadas en la competencia.

In [ ]:
def predict_labels(model, iterator, criterion, fields=fields):

    # Extraemos los vocabularios.
    text_field = fields[0][1]
    nertags_field = fields[1][1]
    tags_vocab = nertags_field.vocab.itos
    words_vocab = text_field.vocab.itos

    model.eval()

    predictions = []

    with torch.no_grad():

        for batch in iterator:

            text_batch = batch.text
            text_batch = torch.transpose(text_batch, 0, 1).tolist()

            # Predecir los tags de las sentences del batch
            predictions_batch = model(batch.text)
            predictions_batch = torch.transpose(predictions_batch, 0, 1)

            # por cada oración predicha:
            for sentence, sentence_prediction in zip(text_batch,
                                                     predictions_batch):
                for word_idx, word_predictions in zip(sentence,
                                                      sentence_prediction):
                    # Obtener el indice del tag con la probabilidad mas alta.
                    argmax_index = word_predictions.topk(1)[1]

                    current_tag = tags_vocab[argmax_index]
                    # Obtenemos la palabra
                    current_word = words_vocab[word_idx]

                    if current_word != '<pad>':
                        predictions.append([current_word, current_tag])
                predictions.append(['EOS', 'EOS'])


    return predictions




In [ ]:
# predictions = predict_labels(model_ft, test_loader, criterion)
# predictions = predict_labels(model_ft, test_iterator_ft, criterion)
# predictions_m3 = predict_labels(model3, test_iterator, criterion3)

# predictions

### **Generar el archivo para la submission**

No hay problema si aparecen unk en la salida. Estos no son relevantes para evaluarlos, usamos solo los tags.

In [ ]:
if (os.path.isfile('./predictions.zip')):
    os.remove('./predictions.zip')

if (not os.path.isdir('./predictions')):
    os.mkdir('./predictions')

else:
    # Eliminar predicciones anteriores:
    shutil.rmtree('./predictions')
    os.mkdir('./predictions')

f = open('predictions/predictions.txt', 'w')
for i, (word, tag) in enumerate(predictions[:-1]):
    if word=='EOS' and tag=='EOS': f.write('\n')
    else: 
      if i == len(predictions[:-1])-1:
        f.write(word + ' ' + tag)
      else: f.write(word + ' ' + tag + '\n')

f.close()

a = shutil.make_archive('predictions', 'zip', './predictions')